Uses the [Census Income Data Set](http://archive.ics.uci.edu/ml/datasets/Census+Income).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
url = 'https://gist.githubusercontent.com/DanielKerrigan/45a3fb3a2bd3d4863d56107f245b20d1/raw/ac994dd75e5a623fcb481f7396a718c9a1ea5900/bank-additional-full.csv'

In [ ]:
df = pd.read_csv(url, sep=';')

In [ ]:
df.rename(columns={"y": "label"}, inplace=True)

In [ ]:
df_train_split, df_test_split = train_test_split(df, test_size=0.25)
df_train = df_train_split.copy()
df_test = df_test_split.copy()

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
num_train = df_train.shape[0]

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_test['label'].replace({ 'no': 0, 'yes': 1 }, inplace=True)
df_train['label'].replace({ 'no': 0, 'yes': 1 }, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df = pd.concat([df_train, df_test])

In [ ]:
df_one_hot = pd.get_dummies(df)
df_train_one_hot = df_one_hot[:num_train]
df_test_one_hot = df_one_hot[num_train:]

In [ ]:
df_train_one_hot.shape[0] == num_train

In [ ]:
X_train = df_train_one_hot.drop(columns=['label']).values
y_train = df_train_one_hot['label'].values

In [ ]:
X_test = df_test_one_hot.drop(columns=['label']).values
y_test = df_test_one_hot['label'].values

In [ ]:
X_train.shape[0] == num_train

In [ ]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_split': [2, 4, 8, 16, 32]
}
clf = GridSearchCV(DecisionTreeClassifier(), parameters, scoring='f1_weighted')
clf.fit(X_train, y_train)

In [ ]:
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
train_preds = np.where(clf.predict(X_train) > 0.5, 1, 0)
(train_preds == y_train).sum() / num_train

In [ ]:
probs = clf.predict(X_test)

In [ ]:
predictions = np.where(probs > 0.5, 1, 0)

In [ ]:
df_test['prediction'] = predictions

In [ ]:
df_test['label'].replace({ 0: 'no term deposit', 1: 'term deposit' }, inplace=True)
df_test['prediction'].replace({ 0: 'no term deposit', 1: 'term deposit' }, inplace=True)

In [ ]:
df_test.to_csv('bank-marketing.csv', index=False, float_format='%.3f')